<a href="https://colab.research.google.com/github/adamMcneil/cs-440-final-project/blob/main/Adam_CS440.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import time
!pip install datasets
from datasets import load_dataset

from transformers import AutoTokenizer, LlamaForSequenceClassification
import torch

from peft import get_peft_model, LoraConfig, TaskType
!pip install transformers
from transformers import Trainer, TrainingArguments

from transformers import BertTokenizer, BertForSequenceClassification

SEED=213

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Helper function to classify one example
def classify_sentiment(test_model, review_text):
    # Tokenize
    inputs = tokenizer(review_text, return_tensors="pt", padding=False, truncation=True).to(device)

    # Forward pass (no gradient needed during inference)
    with torch.no_grad():
        outputs = test_model(**inputs)
        logits = outputs.logits

    # Predicted class
    predicted_class_id = logits.argmax(dim=-1).item()

    # (Optional) Map class ID to label
    id2label = test_model.config.id2label
    predicted_label = id2label[predicted_class_id]

    #if predicted_label == "LABEL_0":
    #    predicted_label = 0
    #else:
    #    predicted_label = 1
    predicted_label = int(predicted_label.split("_")[-1])
    assert predicted_label in [0, 1]

    return predicted_label

In [ ]:
from collections import Counter
import time

def test_model(test_model, validation_dataset):
    print("Running classification test...")
    correct = 0
    total = 0
    start_time = time.time()

    # Track metrics for binary classification (0: negative, 1: positive)
    true_positives = Counter()
    false_positives = Counter()
    false_negatives = Counter()

    for example in validation_dataset:
        review_text = example['text']
        true_label = example['label']
        pred_label = classify_sentiment(test_model, review_text)

        if pred_label == true_label:
            correct += 1
            true_positives[true_label] += 1
        else:
            false_positives[pred_label] += 1
            false_negatives[true_label] += 1

        total += 1

    accuracy = correct / total
    elapsed_time = time.time() - start_time

    print(f"\nAccuracy on {total} examples: {accuracy * 100:.2f}%")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    print(f"Average time per sample: {elapsed_time / total:.4f} seconds")

    labels = sorted(set(true_positives.keys()) | set(false_positives.keys()) | set(false_negatives.keys()))
    for label in labels:
        tp = true_positives[label]
        fp = false_positives[label]
        fn = false_negatives[label]

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        print(f"\nClass {label}:")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")


In [ ]:
# Testing
dataset = load_dataset("stanfordnlp/sst2")
small_train = dataset["train"].select(range(5000))
small_val = dataset["validation"].select(range(872))

test_model(model, small_val)

Running classification test...

Accuracy on 872 examples: 60.89%
Time elapsed: 33.17 seconds
Average time per sample: 0.04 seconds


In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
lora_model = get_peft_model(model, lora_config)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
bert_lora_model = get_peft_model(bert_model, lora_config)

dataset = load_dataset("stanfordnlp/sst2")

small_train = dataset["train"].select(range(10000))
small_val = dataset["validation"].select(range(400))
val = dataset["validation"]

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def preprocess(example):
    return bert_tokenizer(
        example["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
small_train = small_train.map(preprocess, batched=True)
small_val =  small_val.map(preprocess, batched=True)

bert_train = small_train.map(preprocess, batched=True)
bert_val = small_val.map(preprocess, batched=True)

training_args = TrainingArguments(
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val,
)
trainer.train()

test_model(lora_model, val)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.796400
200,0.802100
300,0.802400
400,0.742600
500,0.749400
600,0.750200
700,0.712000
800,0.708000
900,0.696200
1000,0.693300


Running classification test...

Accuracy on 872 examples: 61.01%
Time elapsed: 38.64 seconds
Average time per sample: 0.04 seconds


In [ ]:
trainer = Trainer(
    model=bert_lora_model,
    args=training_args,
    train_dataset=bert_train,
    eval_dataset=bert_val,
)
trainer.train()

test_model(bert_lora_model, validation_dataset)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.661300
200,0.423600
300,0.328500


Running classification test...

Accuracy on 872 examples: 84.98%
Time elapsed: 20.66 seconds
Average time per sample: 0.02 seconds


In [ ]:
test_model(lora_model, trials=100)

Running classification test...

Accuracy on 100 examples: 57.00%
Time elapsed: 6.57 seconds
Average time per sample: 0.07 seconds


In [ ]:
model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Helper function to classify one example
def classify_sentiment(test_model, review_text):
    # Tokenize
    inputs = tokenizer(review_text, return_tensors="pt", padding=False, truncation=True).to(device)

    # Forward pass (no gradient needed during inference)
    with torch.no_grad():
        outputs = test_model(**inputs)
        logits = outputs.logits

    # Predicted class
    predicted_class_id = logits.argmax(dim=-1).item()

    # (Optional) Map class ID to label
    id2label = test_model.config.id2label
    predicted_label = id2label[predicted_class_id]
    predicted_label = int(predicted_label.split("_")[-1])
    assert predicted_label in [0, 1, 2, 3]
    return predicted_label

def test_model(test_model, validation_dataset):
    print("Running classification test...")
    correct = 0
    total = 0
    start_time = time.time()

    for example in validation_dataset:
        review_text = example['text']
        true_label = example['label']

        pred_label = classify_sentiment(test_model, review_text)

        if pred_label == true_label:
            correct += 1
        total += 1

    accuracy = correct / total
    elapsed_time = time.time() - start_time

    # Results
    print(f"\nAccuracy on {total} examples: {accuracy*100:.2f}%")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    print(f"Average time per sample: {elapsed_time/total:.2f} seconds")


In [ ]:
def load_ag_news():
    dataset = load_dataset("ag_news")
    small_train = dataset["train"].shuffle(SEED).select(range(35000))
    small_val = dataset["test"].shuffle(SEED).select(range(1000))
    return dataset["test"], small_train, small_val

In [ ]:
validation_dataset, small_train, small_val = load_ag_news()

test_model(model, validation_dataset.select(range(1000)))

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Running classification test...

Accuracy on 1000 examples: 28.50%
Time elapsed: 69.68 seconds
Average time per sample: 0.07 seconds


In [ ]:
test_model(bert_model, validation_dataset.select(range(1000)))

Running classification test...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
lora_model = get_peft_model(model, lora_config)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
lora_bert_model = get_peft_model(bert_model, lora_config)

validation_dataset, small_train, small_val = load_ag_news()

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def preprocess(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
def bert_preprocess(example):
    return bert_tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
small_train = small_train.map(preprocess, batched=True)
small_val =  small_val.map(preprocess, batched=True)
bert_small_train = small_train.map(bert_preprocess, batched=True)
bert_small_val =  small_val.map(bert_preprocess, batched=True)

training_args = TrainingArguments(
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val,
)
trainer.train()

test_model(lora_model, validation_dataset.select(range(1000)))

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adamwm4 (adamwm4-university-of-illinois-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.793700
200,0.316200
300,0.289400
400,0.271500
500,0.235200
600,0.228400
700,0.206800
800,0.201200
900,0.217900


Running classification test...

Accuracy on 1000 examples: 94.00%
Time elapsed: 46.37 seconds
Average time per sample: 0.05 seconds


In [ ]:
trainer = Trainer(
    model=lora_bert_model,
    args=training_args,
    train_dataset=bert_small_train,
    eval_dataset=bert_small_val,
)
trainer.train()

test_model(lora_bert_model, validation_dataset.select(range(1000)))

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.924500
200,0.429400
300,0.365100
400,0.369900
500,0.352200
600,0.330400
700,0.344100
800,0.308000
900,0.315200
1000,0.255600


Running classification test...

Accuracy on 1000 examples: 24.40%
Time elapsed: 22.20 seconds
Average time per sample: 0.02 seconds


In [ ]:
test_model(lora_bert_model, validation_dataset.select(range(1000)))

Running classification test...


AssertionError: 

In [ ]:
test_model(bert_model, validation_dataset.select(range(1000)))

Running classification test...

Accuracy on 1000 examples: 55.60%
Time elapsed: 24.87 seconds
Average time per sample: 0.02 seconds


In [ ]:
test_model(model, validation_dataset.select(range(1000)))

Running classification test...

Accuracy on 1000 examples: 33.10%
Time elapsed: 78.84 seconds
Average time per sample: 0.08 seconds


In [ ]:
test_model(lora_model, validation_dataset.select(range(1000)))

Running classification test...

Accuracy on 1000 examples: 33.10%
Time elapsed: 73.89 seconds
Average time per sample: 0.07 seconds
